In [40]:
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np 

In [41]:
file_path = "C:/Users/Youcode/Downloads/datascrapperV2-master/appartements_data.csv"


In [42]:
data_frame = pd.read_csv(file_path)

In [43]:
data_frame.head()

,titre,prix,emplacement,type_de_vente,salon,chambre,toilette,surface,lien
0,A louer Appartement prés clinique Badr(Bourgogne),5 500 DH,Casablanca,A louer,1.0,NaN,NaN,100,https://www.avito.ma/fr/bourgogne/appartements...
1,Appartement à vendre 132 m² a bv 2 mars,1 400 000 DH,Casablanca,"Appartements, à vendre",1.0,3.0,2.0,132,https://www.avito.ma/fr/2_mars/appartements/Ap...
2,appartement 3 chambre salon résidence green parck,6 500 DH,Salé,A louer,1.0,NaN,NaN,116,https://www.avito.ma/fr/sala_el_jadida/apparte...
3,Apt H-Standing 160M²/Terrasse/Très Sécurisé/Ra...,11 000 DH,Casablanca,A louer,1.0,NaN,NaN,160,https://www.avito.ma/fr/anfa/appartements/Apt_...
4,Appartement 112 m à mahaj ryad,8 000 DH,Rabat,A louer,1.0,NaN,NaN,112,https://www.avito.ma/fr/hay_riad/appartements/...


In [44]:
data_frame.describe()
data_frame['surface'] = pd.to_numeric(data_frame['surface'], errors='coerce')




In [45]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   titre          431 non-null    object 
 1   prix           431 non-null    object 
 2   emplacement    431 non-null    object 
 3   type_de_vente  431 non-null    object 
 4   salon          272 non-null    float64
 5   chambre        134 non-null    float64
 6   toilette       131 non-null    float64
 7   surface        277 non-null    float64
 8   lien           431 non-null    object 
dtypes: float64(4), object(5)
memory usage: 30.4+ KB


In [56]:
# Convertir toutes les valeurs en chaînes
data_frame['prix'] = data_frame['prix'].astype(str)

# Remplacer les valeurs non spécifiées par NaN
data_frame['prix'] = data_frame['prix'].replace('Prix Non Specifié', np.nan)

# Remplacer les espaces insécables par des espaces normaux
data_frame['prix'] = data_frame['prix'].str.replace('\u202f', ' ', regex=True)

# Extraire uniquement la partie numérique en tenant compte des espaces et des symboles
data_frame['prix'] = data_frame['prix'].str.extract(r'([0-9\s]+)')[0]  # Capturer les chiffres, même avec des espaces

# Nettoyer les espaces et convertir en float
data_frame['prix'] = data_frame['prix'].str.replace(' ', '').astype(float)

<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Youcode\AppData\Local\Temp\ipykernel_20992\2225837716.py:10: SyntaxWarning: invalid escape sequence '\s'
  data_frame['prix'] = data_frame['prix'].str.extract('([0-9\s]+)')[0]  # Capturer les chiffres, même avec des espaces


In [46]:
profile = ProfileReport(data_frame, title="Profiling Report")

In [47]:
# Documenting each column in the dataset as part of the data catalog
data_catalog = pd.DataFrame({
    "Column": data_frame.columns,
    "Description": [
        "Title of the property listing",
        "Price of the property (may include currency)",
        "Location of the property",
        "Type of transaction (e.g., for rent, for sale)",
        "Number of living rooms",
        "Number of bedrooms",
        "Number of bathrooms",
        "Surface area of the property (may include units)",
        "URL link to the listing"
    ],
    "Data Type": data_frame.dtypes.values,
    "Number of Missing Values": data_frame.isna().sum().values,
})

# Calculate preliminary statistics for numerical columns
numeric_data = data_frame.select_dtypes(include='number')
preliminary_stats = numeric_data.describe().T  # transpose for readability

# Calculate additional metrics: median
preliminary_stats['median'] = numeric_data.median()

# Show the catalog and preliminary stats



In [48]:
data_catalog


,Column,Description,Data Type,Number of Missing Values
0,titre,Title of the property listing,object,0
1,prix,Price of the property (may include currency),object,0
2,emplacement,Location of the property,object,0
3,type_de_vente,"Type of transaction (e.g., for rent, for sale)",object,0
4,salon,Number of living rooms,float64,159
5,chambre,Number of bedrooms,float64,297
6,toilette,Number of bathrooms,float64,300
7,surface,Surface area of the property (may include units),float64,154
8,lien,URL link to the listing,object,0


In [49]:
preliminary_stats

,count,mean,std,min,25%,50%,75%,max,median
salon,272.0,1.261029,0.590412,0.0,1.0,1.0,1.0,4.0,1.0
chambre,134.0,2.410448,0.695989,1.0,2.0,2.0,3.0,5.0,2.0
toilette,131.0,2.343511,7.127165,0.0,1.0,2.0,2.0,83.0,2.0
surface,277.0,101.916968,47.957601,1.0,75.0,94.0,120.0,504.0,94.0


In [50]:


# # Sélectionner les colonnes numériques
# columns = ['prix','salon', 'chambre', 'toilette','surface']

# # Calculer la moyenne, la médiane et l'indice d'asymétrie (skewness)
# stats = data_frame[columns].agg(['mean', 'median', 'skew']).T
# stats.columns = ['Moyenne', 'Médiane', 'Skewness']
# print("Statistiques de symétrie :\n", stats)


In [51]:
# Calcul des quartiles et de l'IQR pour les colonnes 'salon' et 'toilette'
Q1_salon = data_frame['salon'].quantile(0.25)
Q3_salon = data_frame['salon'].quantile(0.75)
IQR_salon = Q3_salon - Q1_salon

Q1_toilette = data_frame['toilette'].quantile(0.25)
Q3_toilette = data_frame['toilette'].quantile(0.75)
IQR_toilette = Q3_toilette - Q1_toilette

# Détection des valeurs aberrantes pour 'salon' et 'toilette'
outliers_salon = data_frame[(data_frame['salon'] < (Q1_salon - 1.5 * IQR_salon)) | (data_frame['salon'] > (Q3_salon + 1.5 * IQR_salon))]
outliers_toilette = data_frame[(data_frame['toilette'] < (Q1_toilette - 1.5 * IQR_toilette)) | (data_frame['toilette'] > (Q3_toilette + 1.5 * IQR_toilette))]

# Affichage des résultats
print("Valeurs aberrantes pour 'salon' :\n", outliers_salon[['salon']])
print("\nValeurs aberrantes pour 'toilette' :\n", outliers_toilette[['toilette']])


Valeurs aberrantes pour 'salon' :
      salon
5      2.0
10     2.0
24     2.0
35     2.0
41     2.0
..     ...
407    2.0
413    2.0
414    2.0
425    3.0
427    2.0

[66 rows x 1 columns]

Valeurs aberrantes pour 'toilette' :
      toilette
395      83.0


In [52]:
profile.to_notebook_iframe()
profile.to_html('profile.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

TypeError: ProfileReport.to_html() takes 1 positional argument but 2 were given